# CS246 - Colab 8
## Bloom Filters

### Setup

In this Colab we just need to install a [bloom_filter](https://github.com/hiway/python-bloom-filter), a Python library which offers an implementations of Bloom filters.  Run the cell below!

In [1]:
!pip install bloom_filter

### Data Loading

From the NLTK (Natural Language ToolKit) library, we import a large list of English dictionary words, commonly used by the very first spell-checking programs in Unix-like operating systems.

In [2]:
import nltk
nltk.download('words')

from nltk.corpus import words
word_list = words.words()
print(f'Dictionary length: {len(word_list)}')
print(word_list[:15])

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
Dictionary length: 236736
['A', 'a', 'aa', 'aal', 'aalii', 'aam', 'Aani', 'aardvark', 'aardwolf', 'Aaron', 'Aaronic', 'Aaronical', 'Aaronite', 'Aaronitic', 'Aaru']


Then we load another dataset from the NLTK Corpora collection: ```movie_reviews```.

The movie reviews are categorized between *positive* and *negative*, so we construct a list of words (usually called **bag of words**) for each category.

In [3]:
from nltk.corpus import movie_reviews
nltk.download('movie_reviews')

neg_reviews = []
pos_reviews = []

for fileid in movie_reviews.fileids('neg'):
  neg_reviews.extend(movie_reviews.words(fileid))
for fileid in movie_reviews.fileids('pos'):
  pos_reviews.extend(movie_reviews.words(fileid))

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


### Your task

In this Colab, you will develop a very simplistic spell-checker.  By no means you should think of using it for a real-world use case, but it is an interesting exercise to highlight the strenghts and weaknesses of Bloom Filters!

In [4]:
from bloom_filter import BloomFilter

word_filter = BloomFilter(max_elements=236736)

for word in word_list:
  word_filter.add(word)

word_set = set(word_list)

If you executed the cell above, you now have 3 different variables in your scope:

1.   ```word_list```, a Python list containing the English dictionary (in case insensitive order)
2.   ```word_filter```, a Bloom filter where we have already added all the words in the English dictionary
3.   ```word_set```, a [Python set](https://docs.python.org/3.6/library/stdtypes.html#set-types-set-frozenset) built from the same list of words in the English dictionary

Let's inspect the size of each datastructure using the [getsizeof()](https://docs.python.org/3/library/sys.html#sys.getsizeof) method!



In [6]:
from sys import getsizeof

print(f'Size of word_list (in bytes): {getsizeof(word_list)}')
print(f'Size of word_filter (in bytes): {getsizeof(word_filter)}')
print(f'Size of word_set (in bytes): {getsizeof(word_set)}')



Size of word_list (in bytes): 2115960
Size of word_filter (in bytes): 64
Size of word_set (in bytes): 8388840


You should have noticed how efficient is the Bloom filter in terms of memory footprint!

Now let's find out how fast is the main operation for which we construct Bloom filters: *membership testing*. To do so, we will use the ```%timeit``` IPython magic command, which times the repeated execution of a single Python statement.

In [7]:
%timeit -r 3 "California" in word_list
%timeit -r 3 "California" in word_filter
%timeit -r 3 "California" in word_set

1000 loops, best of 3: 476 µs per loop
The slowest run took 9.61 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 13.8 µs per loop
The slowest run took 687.13 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 79.7 ns per loop


Notice the performance gap between linear search on a list, multiple hash computations in a Bloom filter, and a single hash computation in a native Python ```Set()```.

We now have all the building blocks required to build our spell-checker, and we understand the performance tradeoffs of each datastructure we chose. Write a function that takes as arguments (1) a list of words, and (2) any of the 3 dictionary datastructures we constructed. The function must return the number of words which **do not appear** in the dictionary.

In [32]:
def spell_checker(lst, dictionary):
  """
  Input:  
          (1) list of words
          (2) any of the 3 dictionarty datastructures we contructed
  Output: 
          Returns the number of words which do not appear in the dictionary
  """
  count = 0
  for word in lst:
    if word not in dictionary:
      count += 1
  return count

In [33]:
%timeit spell_checker(neg_reviews, word_filter)
%timeit spell_checker(neg_reviews, word_set)

1 loop, best of 5: 6.15 s per loop
10 loops, best of 5: 86.8 ms per loop


In [34]:
neg_filter = spell_checker(neg_reviews, word_filter)
neg_set = spell_checker(neg_reviews, word_set)

In [35]:
error_rate_filter = neg_filter / (len(neg_reviews))
error_rate_set = neg_set / (len(neg_reviews))

In [36]:
print(error_rate_filter * 100) 
print(error_rate_set * 100)


26.42228481369866
27.987530216064833


Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!

---
# Results
---
*   Q2.1 : 64
*   Q2.2 : 8388840
*   Q3.1 : [1 µs, 100 µs)
*   Q3.2 : [1 ns, 100 ns)
*   Q4.1 : [1 s, 1000 s)
*   Q4.2 : 26.422
*   Q4.3 : [1 ms, 1 s)
*   Q4.4 : 27.988
---
